In [140]:
import pandas as pd
import os
import re


photos = r"C:\Users\mkreidler\Desktop\35w Connect Pics\PHOTOS"

file = r"C:\Users\mkreidler\Desktop\35w Connect Pics\Multisheet_export.xlsx"

df = pd.read_excel(file, sheet_name="Main")

df = df.loc[:, ["name", "db"]]

df["area"] = None

df

def find_area(x):
    pat = r"Area (\d*)"
    num = re.findall(pat, x)
    if num:
        return num[0]
    
df.area = df.name.apply(find_area)
    
areas = df.loc[~df.area.isna(), ["db", "area"]]
    
def map_area(x):
    result =  areas.loc[areas.db == x, "area"].tolist()
    if result:
        return result[0]

df.area = df.db.apply(map_area)

name_area =  df.loc[~df.name.str.contains("Area"), :]

na = name_area.dropna().sort_values("name")

#unique_names = sorted(name_area.name.unique())

styles = na.groupby("name")["area"].apply(set).reset_index()

styles.head()

,name,area
0,3B10,{247}
1,3B11,"{237, 247, 231, 241, 338, 230}"
2,3B11A,"{340, 343, 237, 231, 241, 337, 234}"
3,3B11B,"{340, 343, 237, 231, 241, 337, 234}"
4,3Bn,{347}


In [141]:
import os


photos = r"C:\Users\mkreidler\Desktop\35w Connect Pics\PHOTOS"

pics = []
bases = []
for root, dirs, files in os.walk(photos, topdown = False):
    for name in files:
        full = os.path.join(root, name)
        #print(full)
        pics.append(full)
        base = os.path.basename(name)
        base = os.path.splitext(base)[0]
        num = base.split("_")[0]
        bases.append(num)

        
pictures = pd.DataFrame(zip(pics, bases), columns=["path", "area"])
        
        
pictures = pictures.groupby("area")["path"].apply(set).reset_index()    

#na.groupby("name")["area"].apply(set) 

#save = r"C:\Users\mkreidler\Desktop\pics.xlsx"
#pictures.to_excel(save)

pictures.head()

,area,path
0,228,{C:\Users\mkreidler\Desktop\35w Connect Pics\P...
1,229,{C:\Users\mkreidler\Desktop\35w Connect Pics\P...
2,230,{C:\Users\mkreidler\Desktop\35w Connect Pics\P...
3,231,{C:\Users\mkreidler\Desktop\35w Connect Pics\P...
4,232,{C:\Users\mkreidler\Desktop\35w Connect Pics\P...


In [142]:
pictures.loc[pictures["area"] == "231", "path"].tolist()

[{'C:\\Users\\mkreidler\\Desktop\\35w Connect Pics\\PHOTOS\\231_3B11.jpg',
  'C:\\Users\\mkreidler\\Desktop\\35w Connect Pics\\PHOTOS\\231_3B11A.jpg',
  'C:\\Users\\mkreidler\\Desktop\\35w Connect Pics\\PHOTOS\\231_490.jpg',
  'C:\\Users\\mkreidler\\Desktop\\35w Connect Pics\\PHOTOS\\231_B98.jpg'}]

In [166]:
from slugify import slugify
import shutil

styles = styles.rename(columns={"name":"style"})
base_folder = r"C:\Users\mkreidler\Desktop\35w Connect Pics\Survey Doc by Style"



for i, row in styles.iterrows():
    pics = []
    for a in row.area:
        p = pictures.loc[pictures["area"] == a, "path"].tolist()
        pics.append(p)
    style = slugify(row.style)
    folder = os.path.join(base_folder, style)
    if not os.path.exists(folder):
        os.mkdir(folder)
    else:
        pass
    for pic in pics:
        for pi in pic:
            for p in pi:
                base = os.path.basename(p)
                #print(base)
                newfile = os.path.join(folder, base)
                if not os.path.exists(newfile):
                    shutil.copyfile(p, newfile)


        
